<a href="https://colab.research.google.com/github/tranmanhcuong253/Vietnamese-RAG-Chatbot/blob/main/Vietnamese_RAG_Chatbot_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0.Packages

In [ ]:
# !pip install -qq scrapy
# !pip install langchain
# !pip install -qU langchain-community faiss-cpu
# !pip install -qU langchain-openai
# !pip install --upgrade --quiet  rank_bm25
# !pip install langchain_experimental
# !pip install -U FlagEmbedding

In [ ]:
#Setup OPENAI_API_KEY
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')

#1.Data Crawling

In [ ]:
import requests
import xml.etree.ElementTree as ET
import json

# URLs of the sitemaps
sitemap_urls = [
    'https://hoatuoimymy.com/product-sitemap1.xml',
    'https://hoatuoimymy.com/product-sitemap2.xml'
]

all_urls = []

# Function to fetch and parse XML
def fetch_sitemap(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        root = ET.fromstring(response.content)
        # Extract all <loc> elements that contain the URLs
        for url_element in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
            all_urls.append(url_element.text)
    except Exception as e:
        print(f"Error fetching or parsing {url}: {e}")

# Fetch and parse both sitemaps
for sitemap_url in sitemap_urls:
    fetch_sitemap(sitemap_url)

# Write the URLs to a JSON file
with open('all_urls.json', 'w') as f:
    json.dump(all_urls, f, indent=4)

print(f"Extracted {len(all_urls)} URLs and saved to all_urls.json")


In [ ]:
import pandas as pd
import json
import re
# Specify the filename
filename = '/content/all_urls.json'

# Load the all_urls list from the JSON file
with open(filename, 'r') as file:
    all_urls = json.load(file)

print(f"All URLs loaded from {filename}")


import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup

class CustomSpider(scrapy.Spider):
    name = 'custom_spider'
    start_urls = all_urls

    # Initialize a counter
    request_count = 0

    def parse(self, response):
        self.request_count += 1  # Increment the counter with each request
        description = ""

        # Scraping the review title (h1 tag inside div.product_title)
        review_title = response.css('h1.product-title::text').get()

        if review_title:
            h1_tag = review_title.strip()
        else:
            h1_tag = ""

        # Now, h1_tag contains the content of the h1 tag
        print(h1_tag)


        price = response.css('span.woocommerce-Price-amount')

        if price:
            price = price.get().strip()
        else:
            price = ""
        match = re.search(r'>([\d.,]+₫)<', price)
        if match:
            price = match.group(1)
        # Scraping the ck-content
        ck_contents = response.css('div.woocommerce-Tabs-panel--description')

        for ck_content in ck_contents:
            for element in ck_content.xpath('./*'):
                # Extract the text from h2 and h3 tags
                if element.root.tag == 'h2':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"
                elif element.root.tag == 'h3':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the text from p tags
                elif element.root.tag == 'p':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the list items from ul tags
                elif element.root.tag == 'ul':
                    li_tags = element.css('li')
                    for li_tag in li_tags:
                        description += f"- {' '.join(li_tag.css('::text').getall()).strip()}" + "\n"

        # Initialize an empty array to hold image URLs
        image_urls = []

        # Select all the div elements with the specific class
        image_elements = response.css('div.woocommerce-product-gallery__image')

        # Loop through each image element to extract the URLs
        for element in image_elements:
            # Extract the main image URL from the 'data-large_image' attribute
            image_url = element.css('img::attr(data-large_image)').get()

            # Add the extracted image URL to the array
            if image_url:
                image_urls.append(image_url)

        data = {}

        if h1_tag and description:
            data = {
                "url": response.url,  # Add the URL of the request
                "content": description,
                "price": price,
                "title": h1_tag,  # h1_tag is now guaranteed to be a string
                "image_urls": image_urls
            }

            yield data
        # Print out the current request count
        print('====> h1_tag', h1_tag)
        print('====>description', description)
        print('====>image_urls', image_urls)
        print('====>price', price)

        self.logger.info(f"Number of requests done: {self.request_count}")
        self.logger.info(f"Crawled: {response.url}")


# Initialize the Scrapy crawler process
process = CrawlerProcess({
    'LOG_LEVEL': 'INFO',
    'FEEDS': {
        'output.json': {
            'format': 'json',
            'encoding': 'utf8',
            'store_empty': False,
            'fields': None,
            'indent': 4,
        },
    },
    'CLOSESPIDER_TIMEOUT': 60000000000,  # Close the spider after 60 seconds (adjust as needed)
    'DOWNLOAD_DELAY': 3,  # Delay of 2 seconds between each request
})

# Start the spider
process.crawl(CustomSpider)
process.start()


#2.Data Preprocessing

##2.1.Import JSON data

In [ ]:
import json

# Load the `output.json` file
with open('output.json', 'r',encoding="utf8") as f:
    data = json.load(f)

In [ ]:
# Check the length of the file
len(data)

##2.2.Remove excess characters and duplicate text

In [ ]:
import re
# Remove special characters
for item in data:
  item['content']= re.sub(r'[^\S ]+', ' ', item['content'])

In [ ]:
#Check the data sample format
data[0]

In [ ]:
# The duplicated text of most records
introduction = """Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí: - Hoa tươi mới được nhập về trong ngày - Cập nhật xu hướng hoa mới nhất trên thị trường - Các thiết kế hoa độc lạ và cực kỳ bắt mắt - Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận - Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi - Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách - Hoàn trả tiền khi khách hàng không hài lòng - Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop"""

In [ ]:
# Remove the duplicated text from all records and count the number of records containing that text
count = 0
exception = []
for item in data:
  if introduction in item['content']:
    count = count + 1
    item['content'] = item['content'].replace(introduction,"")
  else:
    exception.append(item)
print(count)

350 out of 351 records have the duplicated text.

In [ ]:
# The exception record which doesn't contain that text.
exception

In [ ]:
# Sample record after removing the duplicated text
data[0]['content']

##2.3.Convert all the records into LangChain's Document format

In [ ]:
from uuid import uuid4
from bs4 import BeautifulSoup
from langchain_core.documents import Document

In [ ]:
list_of_documents = []
for item in data:
  # Extract the price value from HTML tag
  soup = BeautifulSoup(item['price'], 'html.parser')
  price = soup.bdi.text.strip()
  content = item['title'] + ' - Giá tiền: ' + price + ' - ' + item['content']
  # Convert the records to LangChain's Document format and append them to a list
  list_of_documents.append(Document(page_content=content, metadata={"source": item['url'],
                                                                    "image_urls":item['image_urls']}))

In [ ]:
# Convert and append the duplicated text (the introduction) to the list in LangChain's Document format
list_of_documents.append(Document(page_content = introduction))

In [ ]:
# Check the length of the list
len(list_of_documents)

In [ ]:
# Check the list sample
list_of_documents[-1]

##2.4.Semantic Chunking

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for the BAAI/bge-m3 model
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")

In [ ]:
# Using LangChain's SemanticChunker with the percentile strategy and a threshold of 85
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings(model = 'text-embedding-3-small', dimensions = 1024),breakpoint_threshold_type="percentile",breakpoint_threshold_amount=85)

In [ ]:
# Split the documents and count the number of chunks in each document, as well as the number of tokens in each chunk
count = 0
total = 0
list_of_chunks = []
for idx,doc in enumerate(list_of_documents):
  chunks = text_splitter.create_documents([doc.page_content])
  print(f'Number of chunks: {len(chunks)} - Tokens of each chunk',end=' ')
  for chunk in chunks:
      text = chunk.page_content
      tokens = tokenizer.tokenize(text)
      num_tokens = len(tokens)
      if num_tokens > 1:
        total = total + 1
        # Use the parent document index as metadata to retrieve the parent document from the child chunk.
        chunk.metadata['parent'] = idx
        list_of_chunks.append(chunk)
      if num_tokens > 512:
        count = count + 1
      print(num_tokens, end =' ')
  print()
print('Toltal chunks: ',total)
print('Number of chunks which is larger than 512 tokens: ',count)

In [ ]:
# Total Chunks
len(list_of_chunks)

In [ ]:
# Chunk example
list_of_chunks[-1]

In [ ]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(list_of_chunks))]

#3.Retriever Module

##3.1.FAISS Vector Store

In [ ]:
# Load the OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small', dimensions = 1024)

In [ ]:
# Initialize the FAISS vector store
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
# Add all the chunks to the vector store
vector_store.add_documents(documents=list_of_chunks, ids=uuids)

In [ ]:
# Similarity search example with the vector store
semantic_results = vector_store.similarity_search(
    "Hoa hồng",
    k=10,
)
for res in semantic_results:
    print(f"* {res.page_content} [{res.metadata}]")

##3.2.BM25 Retriever

In [ ]:
# Using BM25 retriever from LangChain
from langchain_community.retrievers import BM25Retriever

In [ ]:
# bm25_params = {
#     "k1":1.25,
#     "b":0.5
# }

In [ ]:
# Initialize the BM25 retriever with the configuration to search for the top 10 most relevant results
bm25_retriever = BM25Retriever.from_documents(
  list_of_chunks, k = 10
)

In [ ]:
# Example with BM25 retriever
bm25_results = bm25_retriever.invoke("Hoa hồng")
bm25_results

##3.3.BGE-m3 Reranker

In [ ]:
# Get the result chunks from BM25 Retriever and FAISS vector search
content = set()
retrieval_docs = []

for result in semantic_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

for result in bm25_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

len(retrieval_docs)

In [ ]:
# Use the BAAI/bge-reranker-v2-m3 model to rerank the order of the result chunks based on relevance score
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
pairs = [["Hoa hồng",doc.page_content] for doc in retrieval_docs]
score = reranker.compute_score(pairs,normalize = True)
score

##3.4.Retriever

In [ ]:
# Put all the methods above into a class called 'Retriever'
class Retriever:
  def __init__(self, semantic_retriever, bm25_retriever, reranker):
    self.semantic_retriever = semantic_retriever
    self.bm25_retriever = bm25_retriever
    self.reranker = reranker

  def __call__(self,query):
    semantic_results = self.semantic_retriever.similarity_search(
      query,
      k=10,
    )
    bm25_results = self.bm25_retriever.invoke(query)

    content = set()
    retrieval_docs = []

    for result in semantic_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    for result in bm25_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    pairs = [[query,doc.page_content] for doc in retrieval_docs]

    scores = self.reranker.compute_score(pairs,normalize = True)

    # Retrieve the parent document from the child chunk based on a threshold score.
    context_1 = []
    context_2 = []
    context = []
    parent_ids = set()
    for i in range(len(retrieval_docs)):
      # Relevance score >= 0.6 will be used as context type 1 (indicating higher relevance to the query)
      if scores[i] >= 0.6:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_1.append(list_of_documents[parent_idx])
      # Relevance score >= 0.1 will be used as context type 2 (indicating medium to lower relevance to the query)
      elif scores[i] >= 0.1:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_2.append(list_of_documents[parent_idx])
      # If the relevance score < 0.1, it indicates that there are no relevant documents.
    if len(context_1) > 0:
      print('Context 1')
      context=context_1
    elif len(context_2) > 0:
      print('Context 2')
      context=context_2
    else:
      print('No relevant context')
    return context


In [ ]:
# Test the Retriever
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker)
context = retriever("Hoa hồng")
context

#4.Answer Generator

##4.1.Semantic Router

In [ ]:
flowerSamples = [
    "Tôi muốn mua hoa hồng đỏ để tặng vào ngày kỷ niệm, bạn có mẫu nào đẹp không?",
    "Có loại hoa nào phù hợp để chúc mừng khai trương không?",
    "Tôi cần một lẵng hoa màu vàng để chúc mừng sinh nhật, có loại nào không?",
    "Bạn có hoa cưới dành riêng cho cô dâu không?",
    "Tôi muốn mua hoa để tặng mẹ nhân ngày của Mẹ, có gợi ý gì không?",
    "Hoa nào đẹp và lãng mạn để tặng người yêu vào ngày lễ tình nhân?",
    "Tôi cần hoa để viếng đám tang, có loại nào phù hợp không?",
    "Có bó hoa nào ý nghĩa để chúc mừng tốt nghiệp không?",
    "Bạn có thể giới thiệu một bó hoa ngọt ngào để tặng người yêu không?",
    "Tôi muốn mua hoa để tặng ngày Phụ nữ Việt Nam, bạn có mẫu nào phù hợp không?",
    "Bó hoa nào phù hợp để tri ân thầy cô vào ngày Nhà giáo Việt Nam?",
    "Tôi muốn đặt một giỏ hoa để chúc mừng công ty mới khai trương.",
    "Bạn có hoa lan để làm quà tặng không?",
    "Tôi cần một bó hoa tulip cho ngày kỷ niệm, bạn có loại nào không?",
    "Hoa nào phù hợp để tặng vào dịp Tết Nguyên Đán?",
    "Có bó hoa nào dễ thương cho sinh nhật bé gái không?",
    "Tôi muốn mua giỏ hoa dành tặng bà nhân ngày lễ lớn, có loại nào không?",
    "Bạn có thể làm một bó hoa kiểu cổ điển không?",
    "Tôi cần hoa để trang trí bàn tiệc cưới, bạn có loại nào sang trọng không?",
    "Hoa nào phù hợp để tặng đồng nghiệp trong dịp sinh nhật?",
    "Tôi muốn một bó hoa lãng mạn để tặng vợ nhân kỷ niệm ngày cưới.",
    "Tôi cần hoa tươi để trang trí cho một sự kiện lớn, bạn có loại nào phù hợp không?",
    "Hoa nào phù hợp để gửi tặng người thân trong bệnh viện?",
    "Tôi muốn đặt một lẵng hoa để chúc Tết ông bà, bạn có gợi ý nào không?",
    "Có loại hoa nào phù hợp để tặng sinh nhật bạn mà trông nhẹ nhàng không?",
    "Hoa nào thích hợp để chúc mừng bạn mới sinh con?",
    "Tôi muốn đặt hoa cho lễ kỷ niệm thành lập công ty, có mẫu nào trang trọng không?",
    "Có thể làm một bó hoa theo yêu cầu với màu trắng chủ đạo không?",
    "Tôi muốn mua một bó hoa sen để tặng cho ông bà nhân ngày lễ lớn.",
    "Hoa nào thường dùng để tặng trong dịp lễ Phật Đản?",
    "Tôi cần hoa tặng đồng nghiệp trong lễ kỷ niệm công ty, có loại nào không?",
    "Có bó hoa nào phù hợp để gửi tặng người thân đang đi xa không?",
    "Tôi muốn hoa tươi và đơn giản để tặng cho người bạn thân.",
    "Có loại hoa nào mang ý nghĩa may mắn cho dịp Tết không?",
    "Hoa nào đẹp và ý nghĩa để tặng mẹ vào sinh nhật?",
    "Tôi muốn một bó hoa với tông màu hồng để tặng bạn gái, bạn có gợi ý gì không?",
    "Có loại hoa nào dễ bảo quản để làm quà không?",
    "Tôi muốn đặt hoa cho lễ hội mùa xuân, có loại nào phù hợp không?",
    "Hoa nào thích hợp để tặng vào lễ Phật Đản cho người lớn tuổi?",
    "Tôi cần một giỏ hoa để chúc mừng bạn mở cửa hàng mới.",
    "Có bó hoa nào độc đáo để chúc mừng sinh nhật đồng nghiệp không?",
    "Hoa nào đẹp để tặng vợ nhân dịp ngày Phụ nữ?",
    "Tôi muốn đặt hoa cho buổi họp mặt cuối năm, bạn có gợi ý gì không?",
    "Có loại hoa nào tượng trưng cho sự thành công và phát đạt không?",
    "Tôi cần một bó hoa dễ thương để chúc mừng bạn mới thăng chức.",
    "Hoa nào ý nghĩa để tặng bố nhân dịp sinh nhật?",
    "Tôi muốn hoa phù hợp để gửi lời chia buồn, bạn có loại nào không?",
    "Có hoa nào nhẹ nhàng để chúc mừng sinh nhật bé trai không?",
    "Tôi muốn tặng hoa cho cô giáo nhân ngày 20/11, bạn có mẫu nào không?",
    "Hoa nào thích hợp để gửi tặng dịp giáng sinh?",
    "Tôi cần hoa để trang trí không gian hội nghị, bạn có loại nào sang trọng không?",
    "Hoa nào phù hợp để tặng đối tác trong ngày khai trương công ty?",
    "Tôi muốn hoa lan để trưng bày trong dịp Tết Nguyên Đán.",
    "Hoa nào thích hợp để tặng cho em gái nhân dịp sinh nhật?",
    "Tôi cần một bó hoa mang màu sắc tươi sáng cho ngày lễ Phục sinh.",
    "Có loại hoa nào đặc biệt để tặng ông bà nhân dịp mừng thọ không?",
    "Tôi muốn đặt hoa tặng người yêu nhân dịp sinh nhật, có mẫu nào không?",
    "Hoa nào ý nghĩa để chúc mừng đồng nghiệp thăng chức?",
    "Tôi cần hoa để chúc mừng mẹ nhân ngày lễ Quốc tế Phụ nữ.",
    "Có loại hoa nào phù hợp để gửi tặng bạn cũ nhân dịp gặp lại không?",
    "Tôi muốn một giỏ hoa thanh nhã để tặng người lớn tuổi nhân dịp Tết.",
    "Hoa nào đẹp để gửi lời cám ơn đến sếp?",
    "Tôi muốn hoa lan cho sinh nhật bà ngoại, có mẫu nào không?",
    "Có hoa nào dễ chăm sóc để tặng người thân ở bệnh viện không?",
    "Tôi cần hoa để trang trí cho lễ hội đêm giao thừa, bạn có loại nào không?",
    "Hoa nào thích hợp để tặng cho bố nhân dịp lễ Cha?",
    "Tôi muốn đặt hoa theo yêu cầu với sắc đỏ chủ đạo để tặng vào Valentine.",
    "Có bó hoa nào đặc biệt để tặng bạn thân nhân dịp sinh nhật?",
    "Tôi muốn đặt hoa mừng ngày Nhà giáo, có loại nào trang nhã không?",
    "Hoa nào thích hợp để tặng trong lễ kỷ niệm 10 năm ngày cưới?",
    "Có loại hoa nào ý nghĩa để gửi lời xin lỗi không?",
    "Tôi muốn một bó hoa rực rỡ để tặng bạn gái, có mẫu nào không?",
    "Hoa nào phổ biến để gửi tặng người thân trong ngày lễ Halloween?",
    "Tôi cần hoa tặng đồng nghiệp nữ, có mẫu nào nữ tính không?",
    "Có bó hoa nào dành riêng cho dịp tốt nghiệp không?",
    "Tôi muốn hoa để trang trí phòng khách dịp Tết, có loại nào sang trọng không?",
    "Hoa nào ý nghĩa để tặng người thân nhân dịp năm mới?",
    "Tôi muốn đặt hoa cho lễ mừng thọ ông bà, bạn có gợi ý gì không?",
    "Có loại hoa nào đặc biệt dành cho ngày Valentine không?",
    "Tôi cần hoa để tặng người mới khỏi bệnh, có loại nào không?",
    "Hoa nào đẹp và ý nghĩa để gửi tặng thầy cô nhân ngày Nhà giáo?",
    "Tôi muốn hoa đơn giản để gửi tặng bạn bè vào dịp lễ Quốc tế Lao động.",
    "Có bó hoa nào dễ thương để chúc mừng bạn vừa sinh con gái không?",
    "Hoa nào ý nghĩa để tặng nhân dịp kỷ niệm 20 năm ngày cưới?",
    "Tôi cần hoa tặng đồng nghiệp trong lễ chia tay, bạn có mẫu nào không?",
    "Hoa nào mang thông điệp hy vọng để tặng người đang trải qua khó khăn?",
    "Tôi muốn một bó hoa trắng cho lễ tốt nghiệp, có loại nào không?",
    "Có loại hoa nào thích hợp để chúc mừng bạn chuyển nhà không?",
    "Tôi muốn mua hoa để tặng nhân viên vào dịp cuối năm, bạn có gợi ý không?",
    "Hoa nào đặc biệt để tặng bạn gái nhân kỷ niệm 1 năm quen nhau?",
    "Tôi cần hoa để chúc mừng ngày thành lập công ty, bạn có loại nào không?",
    "Hoa nào phù hợp để tặng dịp lễ Tạ ơn?",
    "Tôi muốn đặt hoa cho lễ giáng sinh, có mẫu nào phù hợp không?",
    "Có loại hoa nào phù hợp để trang trí cho lễ hội mùa hè không?",
    "Tôi muốn một bó hoa dễ bảo quản để tặng bà ngoại.",
    "Hoa nào thường dùng để chúc mừng bạn thăng chức?",
    "Tôi muốn hoa nhẹ nhàng để gửi lời cám ơn mẹ nhân dịp đặc biệt.",
    "Hoa nào đẹp để tặng vợ vào lễ kỷ niệm kết hôn?",
    "Tôi cần hoa tặng người mới sinh em bé, bạn có mẫu nào dễ thương không?",
    "Có hoa nào tượng trưng cho tình bạn để tặng bạn thân không?"
]


In [ ]:
chatSamples =  [
    "Chào bạn! Hôm nay thời tiết thế nào nhỉ?",
    "Mùa này có gì thú vị không bạn?",
    "Hello shop",
    "Chàp shop",
    "Chào bạn! Hôm nay bạn thế nào?",
    "Bạn thích làm việc ở đây không?",
    "Bạn có thích mùa hè không?",
    "Bạn có thể kể một câu chuyện vui không?",
    "Bạn có nghe loại nhạc nào không?",
    "Bạn nghĩ gì về việc đọc sách vào buổi sáng?",
    "Bạn thích ăn món gì nhất?",
    "Bạn có kế hoạch gì cho cuối tuần không?",
    "Bạn có thích thể thao không?",
    "Bạn có hay du lịch không?",
    "Bạn thích màu gì nhất?",
    "Bạn có câu châm ngôn sống nào không?",
    "Có nơi nào bạn muốn ghé thăm không?",
    "Bạn thích hoạt động ngoài trời không?",
    "Có quyển sách nào bạn muốn giới thiệu không?",
    "Bạn có lời khuyên gì cho một cuộc sống hạnh phúc không?",
    "Bạn có thói quen nào vào buổi sáng không?",
    "Bạn có thú cưng không?",
    "Bạn thích âm nhạc loại nào?",
    "Bạn thích mùa nào nhất trong năm?",
    "Bạn có hay xem phim không?",
    "Bạn có tin vào may mắn không?",
    "Bạn có đam mê nào khác không?",
    "Có điều gì đặc biệt mà bạn muốn học không?",
    "Bạn có lời khuyên nào để có năng lượng tích cực không?",
    "Bạn thích làm gì vào thời gian rảnh?",
    "Có bài hát nào bạn yêu thích không?",
    "Bạn có thói quen tập thể dục không?",
    "Bạn có thể kể về một ngày thú vị của mình không?",
    "Bạn đã bao giờ tham gia sự kiện nào lớn chưa?",
    "Có bộ phim nào mà bạn không bao giờ quên không?",
    "Bạn có thích làm việc cùng đồng nghiệp không?",
    "Bạn có mơ ước nào chưa thực hiện không?",
    "Bạn nghĩ gì về cuộc sống thành thị?",
    "Bạn đã học được gì khi làm việc ở đây?",
    "Bạn có người thân nào cũng làm nghề này không?",
    "Có món nào bạn hay ăn vào mùa hè không?",
    "Bạn có thích mùa đông không?",
    "Có khi nào bạn gặp khách hàng khó tính không?",
    "Bạn có dự định học thêm gì không?",
    "Có gì đặc biệt bạn muốn chia sẻ hôm nay không?",
    "Bạn thích ngắm hoàng hôn hay bình minh?",
    "Bạn nghĩ sao về phong cách sống tối giản?",
    "Có sở thích nào bạn muốn khám phá không?",
    "Bạn có thích làm việc nhóm không?",
    "Bạn có hay chụp ảnh không?",
    "Bạn có tin vào số mệnh không?",
    "Bạn nghĩ gì về việc sống ở vùng quê?",
    "Có trò chơi nào bạn hay chơi không?",
    "Bạn có lời khuyên nào về sức khỏe không?",
    "Bạn có thích nấu ăn không?",
    "Có địa điểm nào bạn muốn đến thử không?",
    "Bạn có thói quen gì vào buổi tối không?",
    "Có món nào bạn đặc biệt ghét không?",
    "Bạn thích tham gia sự kiện nào nhất?",
    "Có bài hát nào làm bạn vui không?",
    "Bạn có thích sưu tầm gì không?",
    "Bạn có thích trồng cây không?",
    "Bạn có lời khuyên nào về việc giảm căng thẳng không?",
    "Bạn có thích làm việc ở đây lâu dài không?",
    "Có bộ phim nào bạn xem lại nhiều lần không?",
    "Bạn thích làm gì vào cuối tuần?",
    "Bạn có tin vào tâm linh không?",
    "Bạn có thể chia sẻ một câu nói yêu thích không?",
    "Có điều gì thú vị về công việc này không?",
    "Bạn có thường đi xem triển lãm nghệ thuật không?",
    "Bạn có nghĩ mình sẽ chuyển nghề không?",
    "Bạn thích đồ uống gì nhất?",
    "Bạn có thói quen viết nhật ký không?",
    "Bạn nghĩ sao về du lịch một mình?",
    "Có trò nào bạn thích chơi từ nhỏ không?",
    "Bạn có thường gặp khách du lịch không?",
    "Bạn có thể chia sẻ bí quyết thành công không?",
    "Bạn nghĩ sao về việc tự kinh doanh?",
    "Bạn có thích đi phượt không?",
    "Có món ăn nào bạn không thể cưỡng lại không?",
    "Bạn nghĩ gì về công việc tình nguyện?",
    "Bạn có sở thích nào mới không?",
    "Bạn có thích chụp ảnh phong cảnh không?",
    "Bạn có thể kể một bí mật nhỏ không?",
    "Có nơi nào bạn muốn quay lại nhiều lần không?",
    "Bạn có thể chia sẻ điều gì đó vui nhộn không?",
]


In [ ]:
len(chatSamples),len(flowerSamples)

In [ ]:
from typing import List
class Route():
    def __init__(
        self,
        name: str = None,
        samples:List = []
    ):

        self.name = name
        self.samples = samples

In [ ]:
import numpy as np

class SemanticRouter():
    def __init__(self, routes, embedding = embeddings):
        self.routes = routes
        self.embedding = embedding
        self.routesEmbedding = {}

        for route in self.routes:
            self.routesEmbedding[
                route.name
            ] = self.embedding.embed_documents(route.samples)

    def get_routes(self):
        return self.routes

    def guide(self, query):
        queryEmbedding = self.embedding.embed_query(query)
        queryEmbedding = queryEmbedding / np.linalg.norm(queryEmbedding)
        scores = []

        # Calculate the cosine similarity of the query embedding with the sample embeddings of the router.

        for route in self.routes:
            routesEmbedding = self.routesEmbedding[route.name] / np.linalg.norm(self.routesEmbedding[route.name])
            score = np.mean(np.dot(routesEmbedding, queryEmbedding.T).flatten())
            scores.append((score, route.name))

        scores.sort(reverse=True)
        return scores

In [ ]:
# Test the Semantic Router
flowerRoute = Route(name = 'flower', samples = flowerSamples)
chatRoute = Route(name = 'chat', samples = chatSamples)
semanticRoute = SemanticRouter(routes = [flowerRoute, chatRoute])

In [ ]:
testSamples = [
    "Tôi muốn mua hoa hồng đỏ để tặng vào ngày kỷ niệm, bạn có mẫu nào đẹp không?",
    "Có loại hoa nào phù hợp để chúc mừng khai trương không?",
    "Tôi cần một lẵng hoa màu vàng để chúc mừng sinh nhật, có loại nào không?",
    "Shop ơi cho mình hỏi một chút",
    "Hello shop",
    "Xin chào, hôm nay bạn thế nào",
    "Tớ muốn tâm sự với cậu một chút",
    "Hi Shop, mình muốn tư vấn"
]

In [ ]:
for sample in testSamples:
  route = semanticRoute.guide(sample)
  print(route)

##4.2.Reflection

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
class Reflection():
    def __init__(self, llm):
        self.llm = llm


    def __call__(self, chatHistory, lastItemsConsidereds=100):

        if len(chatHistory) >= lastItemsConsidereds:
            chatHistory = chatHistory[len(chatHistory) - lastItemsConsidereds:]

        history_string = "\n".join(
            [
                f"{message['role']} : {message['content']}"
                for message in chatHistory[:-1]
            ]
        )

        chat_template = PromptTemplate.from_template(
            """Given a chat history and the latest user question which might reference context in the chat history,
        formulate a standalone question in Vietnamese which can be understood without the chat history.
        Do NOT answer the question, just reformulate it if needed and otherwise return it as is.

        Chat History:
        {history_string}

        Latest User Question:
        {input}"""
        )

        chain = chat_template | self.llm
        response = chain.invoke({"input": chatHistory[-1]["content"], "history_string": history_string})

        return response.content

In [ ]:
# Test the Reflection
llm = ChatOpenAI(model="gpt-4o",temperature = 1.0)
reflection = Reflection(llm)

In [ ]:
chatHistory = [{"role": "user", "content": "Xin chào shop"},
               {"role": "assistant", "content": "Chào bạn tôi có thể giúp gì cho bạn"},
               {"role": "user", "content": "Tôi cần tư vấn một số lẵng hoa 2 tầng"},
               {"role": "assistant", "content":"Bạn muốn tư vấn lẵng hoa với hoa gì?"},
               {"role": "user", "content": "Hoa hồng"}]


In [ ]:
history_string = "\n".join(
    [
        f"{message['role']} : {message['content']}"
        for message in chatHistory[:-1]
    ]
)
history_string

In [ ]:
reflection(chatHistory)

##4.3.LLM Generator

In [ ]:
# Initialize the OpenAI GPT-4 mini model with an answer prompt and combine them into a chain
answerModel = ChatOpenAI(model="gpt-4o-mini",temperature = 0.5)

In [ ]:
answerPrompt = PromptTemplate.from_template("""
    Hãy trở thành chuyên gia tư vấn bán hàng cho một cửa hàng bán lẵng hoa.
    Câu hỏi của khách hàng: {query}\nTrả lời câu hỏi dựa vào các thông tin sản phẩm dưới đây: {source_information}.
""")

In [ ]:
answerChain = answerPrompt | answerModel

#4'.Agent Tools (using for RAG version 2)

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from typing import Optional, Type, List

In [ ]:
# Define a BaseModel Retrieve class for LLM tool calling
class Retrieve(BaseModel):
    """
    Searches the knowledge base for answers. The query parameter should contain the contextualized question to search for in the knowledge base.
    """
    query: str = Field(description="should be a search query")


In [ ]:
# The toolPrompt version 1
toolPrompt = PromptTemplate.from_template("""
    You are an AI assistant of a flower shop.
    \\nYou do not know anything about the information and operating procedures of the shop,
    must not answer the user directly, instead *** use the tools to get the most accurate information.***
    \\nIf user just wanna say hello, thank you, normal talk or ask for some life advice, general knowledge, you can reply.
    \\n However, in any other situation, you must use the tool \'Retrieve\' without exception.
    \\n Your customers are Vietnamese, so always reply in Vietnamese.
    \\n Here is your chat history: {chat_history}
""")

In [ ]:
# The toolPrompt version 2
toolPrompt = PromptTemplate.from_template("""
    You are an AI assistant of a flower shop.
    For any questions regarding flower products, you ***must use the `Retrieve` tool*** to obtain accurate information.
    To use the `Retrieve` tool, take the user's most recent question as well as relevant chat history, extract a clear, concise search query from the question and chat context. Pass this query to the `Retrieve` tool by setting the `query` parameter.
    For all other questions or general interactions, such as greetings, expressions of gratitude, or requests for life advice or general knowledge, you may reply directly.
    \\n Your customers are Vietnamese, so always reply in Vietnamese.
    \\n Here is your chat history: {chat_history}
""")

In [ ]:
# Initialize a LLM with tool
llm = ChatOpenAI(model='gpt-4o',temperature = 0.5)
agent = llm.bind_tools([Retrieve])
agentChain = toolPrompt | agent

In [ ]:
# Test the tool calling with a conversation about Flower products
flowerChatHistory = [{"role": "user", "content": "Xin chào shop"},
               {"role": "assistant", "content": "Chào bạn tôi có thể giúp gì cho bạn"},
               {"role": "user", "content": "Tôi cần tư vấn một số lẵng hoa 2 tầng"},
               {"role": "assistant", "content":"Bạn muốn tư vấn lẵng hoa với hoa gì?"},
               {"role": "user", "content": "Hoa hồng"}]
flower_history_string = "\n".join(
    [
        f"{message['role']} : {message['content']}"
        for message in flowerChatHistory
    ]
)
flower_history_string

In [ ]:
response = agentChain.invoke({"chat_history":flower_history_string})

In [ ]:
# Format of a tool calling response
response.additional_kwargs['tool_calls']

In [ ]:
# Test the tool calling with a normal, chitchat conversation
chatHistory = [{"role": "user", "content": "Xin chào shop"},
               {"role": "assistant", "content": "Chào bạn tôi có thể giúp gì cho bạn"},
               {"role": "user", "content": "Quốc kỳ Việt Nam có bao nhiêu màu?"}]
history_string = "\n".join(
    [
        f"{message['role']} : {message['content']}"
        for message in chatHistory
    ]
)
history_string

In [ ]:
agentChain.invoke({"chat_history":history_string})

#5.API

In [ ]:
!pip install pyngrok
!pip install flask_cors

In [ ]:
import os
import json
from google.colab import userdata
from pyngrok import ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS

In [ ]:
# Replace 'YOUR_NGROK_AUTHTOKEN' with the authtoken you copied from the ngrok dashboard
authtoken = userdata.get("ngrok_key")
ngrok.set_auth_token(authtoken)

app = Flask(__name__)
CORS(app)  # Apply CORS to the Flask app

@app.route('/v1/chat', methods=['POST'])
def chat_v1():
    # Extract parameters from the request
    user_message = request.json.get('message', {})
    context = request.json.get('context', [])
    stream = True  # Default to False if not provided

    print(f'Message: {user_message}')
    print(f'Context: {context}')

    query = reflection(context)
    print(f'REFINED: {query}')
    route = semanticRoute.guide(query)[0][1]

    if route == 'flower':
      print('Guide to FLOWER')
      # refine_query = reflection(chatHistory)
      context = retriever(query)
      source_information = ""
      for doc in context:
        content = doc.page_content + ' - Link ảnh: #' + str(doc.metadata['image_urls'])
        source_information+= content + "\n"
      if stream:
        def generate():
          for chunk in answerChain.stream({"query": query, "source_information": source_information}):
            yield chunk.content
        return app.response_class(generate(), mimetype='text/plain')
      else:
        reponse = answerChain.invoke({"query": query, "source_information": source_information})
        return jsonify({'response': reponse.content})
    else:
      print('Guide to CHAT')
      if stream:
        def generate():
          for chunk in answerModel.stream(query):
            yield chunk.content
        return app.response_class(generate(), mimetype='text/plain')
      else:
        reponse = answerModel.invoke(query)
        return jsonify({'response': reponse.content})

@app.route('/v2/chat', methods=['POST'])
def chat_v2():
    # Extract parameters from the request
    user_message = request.json.get('message', {})
    context = request.json.get('context', [])
    stream = True  # Default to False if not provided

    print(f'Message: {user_message}')
    print(f'Context: {context}')

    history_string = "\n".join(
      [
          f"{message['role']} : {message['content']}"
          for message in context
      ]
    )
    agent_response = agentChain.invoke({"chat_history":history_string})
    if 'tool_calls' in agent_response.additional_kwargs:
      print('Guide to FLOWER')
      refine_query = agent_response.tool_calls[0]['args']['query']
      print(f'REFINED: {refine_query}')
      context = retriever(refine_query)
      source_information = ""
      for doc in context:
        content = doc.page_content + ' - Link ảnh: #' + str(doc.metadata['image_urls'])
        source_information+= content + "\n"

      if stream:
        def generate():
          for chunk in answerChain.stream({"query": refine_query, "source_information": source_information}):
            yield chunk.content
        return app.response_class(generate(), mimetype='text/plain')
      else:
        reponse = answerChain.invoke({"query": refine_query, "source_information": source_information})
        return jsonify({'response': reponse.content})
    else:
      print('Guide to CHAT')
      if stream:
        def generate():
          for chunk in agent_response.content.split(" "):
            yield chunk + " "
        return app.response_class(generate(), mimetype='text/plain')
      else:
        return jsonify({'response': agent_response.content})

if __name__ == '__main__':
    # Start ngrok to tunnel the Flask app
    url = ngrok.connect(5000)
    print(f" * ngrok tunnel: {url}")

    # Start Flask app
    app.run(port=5000)